In [5]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from gurobipy import GRB, Model

In [6]:
static = pd.read_csv("csv_files/static_with_t.csv")
detected = pd.read_csv("csv_files/detection_graph.csv")

In [7]:
level_order = [
    "root_cross_right",
    "root_cross_left",
    "root_cross_right2",
    "conn_k",
    "cross_e",
    "cross_bag_br",
    "cross_ij1",
    "cross_e_c",
    "cross_f",
    "bag_b_r",
    "bag_a_r",
    "conn_b",
    "cross_ij2",
    "conn_e",
    "conn_c",
    "cross_k",
    "cross_f30",
    "conn_i",
    "conn_j",
    "box",
    "conn_a",
    "conn_f_30",
    "cross_f37",
    "conn_f_60",
    "bag_b_l",
    "cross_gh",
    "conn_f_37",
    "conn_d",
    "conn_g",
    "conn_h",
    "bag_a_l",
]

In [8]:
quadratic_model = Model("quadratic")

Restricted license - for non-production use only - expires 2024-10-28


In [9]:
variables = {}
for name in level_order:
    variables[f"{name}_x"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_x"
    )
    variables[f"{name}_y"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_y"
    )
    variables[f"{name}_z"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_z"
    )

In [10]:
obj_fn = 0
for name in level_order:
    if name == "root":
        continue
    node = detected[detected["name"] == name]
    if node.shape[0] != 0:
        detected_x = node["x"].values[0]
        detected_y = node["y"].values[0]
        detected_z = node["z"].values[0]
        term = (
            (variables[f"{name}_x"] - detected_x) ** 2
            + (variables[f"{name}_y"] - detected_y) ** 2
            + (variables[f"{name}_z"] - detected_z) ** 2
        )
        obj_fn += term

In [11]:
# for name in level_order:
#     node_info = static[static["name"] == name]
#     parent = node_info["parent"].values[0]
#     distance_to_parent = node_info["distance_to_parent"].values[0]
#     if parent == "root":
#         quadratic_model.addQConstr(
#             (
#                 variables[f"{name}_x"] ** 2
#                 + variables[f"{name}_y"] ** 2
#                 + variables[f"{name}_z"] ** 2
#             )
#             <= distance_to_parent**2
#         )
#     else:
#         quadratic_model.addQConstr(
#             (
#                 (variables[f"{name}_x"] - variables[f"{parent}_x"]) ** 2
#                 + (variables[f"{name}_y"] - variables[f"{parent}_y"]) ** 2
#                 + (variables[f"{name}_z"] - variables[f"{parent}_z"]) ** 2
#             )
#             <= distance_to_parent**2
#         )

In [12]:
for name in level_order:
    node_info = static[static["name"] == name]
    parent = node_info["parent"].values[0]
    distance_to_parent = node_info["distance_to_parent"].values[0]
    if name == "root_cross_left":
        quadratic_model.addQConstr(variables[f"{name}_x"] == -13)
        quadratic_model.addQConstr(variables[f"{name}_y"] == -20)
        quadratic_model.addQConstr(variables[f"{name}_z"] == 0)

    elif name == "root_cross_right":
        quadratic_model.addQConstr(variables[f"{name}_x"] == 10)
        quadratic_model.addQConstr(variables[f"{name}_y"] == -1)
        quadratic_model.addQConstr(variables[f"{name}_z"] == 0)
    else:
        quadratic_model.addQConstr(
            (
                (variables[f"{name}_x"] - variables[f"{parent}_x"]) ** 2
                + (variables[f"{name}_y"] - variables[f"{parent}_y"]) ** 2
                + (variables[f"{name}_z"] - variables[f"{parent}_z"]) ** 2
            )
            <= distance_to_parent**2
        )

In [13]:
quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)

In [14]:
quadratic_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 93 columns and 6 nonzeros
Model fingerprint: 0x7da35b23
Model has 54 quadratic objective terms
Model has 29 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+00]
  Objective range  [6e+00, 4e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+02, 2e+02]
  RHS range        [1e+00, 2e+01]
  QRHS range       [2e+01, 1e+04]
Presolve removed 6 rows and 6 columns
Presolve time: 0.02s
Presolved: 234 rows, 202 columns, 403 nonzeros
Presolved model has 30 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.419e+04
 Factor NZ  : 1.502e+04
 Factor Ops : 1.585e+06 (less than 1 second per iteration)
 Threads    : 1

                

In [15]:
# print all variable values
# for v in quadratic_model.getVars():
#     print(v.varName, v.x)

In [16]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from transformation_utils import (
    get_rotation,
    rotate_graph,
    get_rotation_nn,
    optim_rotation,
)

G1 = nx.Graph()
for i in range(len(static)):
    G1.add_node(static['name'][i], pos=(static['x'][i], static['z'][i], static['y'][i]))
for i in range(len(static)):
    if static['parent'][i] != 0 and static['parent'][i] != '0':
        G1.add_edge(static['name'][i], static['parent'][i])


G2 = nx.Graph()
for name in level_order:
    # if x and y and z are  different than 250 or -250
    if variables[f"{name}_x"].x != 250 and variables[f"{name}_x"].x != -250:
        # print(name, variables[f"{name}_x"].x, variables[f"{name}_y"].x, variables[f"{name}_z"].x)
        G2.add_node(
            name,
            pos=(
                variables[f"{name}_x"].x,
                variables[f"{name}_z"].x,
                variables[f"{name}_y"].x,
            ),
        )




plot_graph(static_graph=G1, detected_graph=G2, name="qcqp2", save=True)

Graph saved to qcqp2.html


In [17]:
# print cross_bag_br_x
print(variables["root_cross_right_x"].x)

10.0
